# Starbucks Promotion Analysis

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer **accumulates at least 10 dollars** in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. **You'll want to try to assess what a certain demographic group will buy when not receiving any offers.**

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example

(1) You could build a machine learning model that predicts how much someone will spend based on demographics and offer type.  
  
(2) You could build a model that predicts whether or not someone will respond to an offer.   
  
(3) You could develop a set of heuristics that determine what offer you should send to each customer. (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

# Overview  of this notebook

1. Load in files
2. Clean the data
3. Univariate Exploration
4. 

# 1. Load in files

In [1]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import seaborn as sns
import time
% matplotlib inline

# read in the json files
portfolio = pd.read_csv('data/portfolio_clean.csv', )
profile = pd.read_csv('data/profile_clean.csv')
transcript = pd.read_csv('data/transcript_clean.csv')
interaction = pd.read_csv('data/interaction.csv')

pd.options.display.max_rows = 100
pd.set_option("display.max_columns", 500)

In [2]:
portfolio.head()

,channels,difficulty,duration,id,offer_type,reward
0,"['email', 'mobile', 'social']",10,7,bogo_7days$10,bogo,10
1,"['web', 'email', 'mobile', 'social']",10,5,bogo_5days$10,bogo,10
2,"['web', 'email', 'mobile']",0,4,info_4days,informational,0
3,"['web', 'email', 'mobile']",5,7,bogo_7days$5,bogo,5
4,"['web', 'email']",20,10,disc_10days$20-$5,discount,5


In [3]:
profile.head()

,age,became_member_on,id,income,gender_F,gender_M,gender_O
0,NaN,2017-02-12,68be06ca386d4c31939f3a4f0e3dd783,NaN,NaN,NaN,NaN
1,55.0,2017-07-15,0610b486422d4921ae7d2bf64640c50b,112000.0,1.0,0.0,0.0
2,NaN,2018-07-12,38fe809add3b4fcf9315a9694bb96ff5,NaN,NaN,NaN,NaN
3,75.0,2017-05-09,78afa995795e4d85b5d9ceeca43f5fef,100000.0,1.0,0.0,0.0
4,NaN,2017-08-04,a03223e636434f42ac4c3df47e8bac43,NaN,NaN,NaN,NaN


In [4]:
transcript.head()

,event,person,time,transaction,id
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,NaN,bogo_7days$5
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,NaN,disc_10days$20-$5
2,offer received,e2127556f4f64592b11af22de27a7932,0,NaN,disc_7days$10-$2
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,NaN,disc_10days$10-$2
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,NaN,bogo_5days$10


In [5]:
interaction.head()

,person,age,income,gender_F,gender_M,gender_O,became_year,became_month_sin,became_month_cos,became_day_sin,became_day_cos,became_dow_sin,became_dow_cos,offer_bogo,offer_disc,difficulty,duration,reward,email,mobile,social,web,t_received,t_viewed,amt_till_viewed,t_completed,amt_till_completed
0,78afa995795e4d85b5d9ceeca43f5fef,75.0,100000.0,1.0,0.0,0.0,2017,0.500000,-8.660254e-01,0.968077,-0.250653,0.781831,0.623490,1,0,5,7,5,1,1,0,1,0,6.0,0.0,132.0,19.89
1,a03223e636434f42ac4c3df47e8bac43,NaN,NaN,NaN,NaN,NaN,2017,-0.866025,-5.000000e-01,0.724793,0.688967,-0.433884,-0.900969,0,1,20,10,5,1,0,0,1,0,6.0,0.0,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,68.0,70000.0,0.0,1.0,0.0,2018,0.866025,-5.000000e-01,-0.848644,0.528964,0.433884,-0.900969,0,1,10,7,2,1,1,0,1,0,18.0,0.0,NaN,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,NaN,NaN,NaN,NaN,NaN,2017,-1.000000,-1.836970e-16,-0.937752,0.347305,0.000000,1.000000,0,1,10,10,2,1,1,1,1,0,12.0,0.0,NaN,NaN
4,68617ca6246f4fbc85e91a2a49552598,NaN,NaN,NaN,NaN,NaN,2017,-0.866025,5.000000e-01,0.394356,0.918958,0.000000,1.000000,1,0,10,5,10,1,1,1,1,0,84.0,0.0,NaN,NaN


In [6]:
offer_viewed = pd.read_csv("data/firstly_viewed_offers.csv")
print(offer_viewed.shape)
offer_viewed.head()

(32007, 27)


,person,age,income,gender_F,gender_M,gender_O,became_year,became_month_sin,became_month_cos,became_day_sin,became_day_cos,became_dow_sin,became_dow_cos,offer_bogo,offer_disc,difficulty,duration,reward,email,mobile,social,web,t_received,t_viewed,amt_till_viewed,t_completed,amt_till_completed
0,78afa995795e4d85b5d9ceeca43f5fef,75.0,100000.0,1.0,0.0,0.0,2017,0.500000,-8.660254e-01,0.968077,-0.250653,0.781831,0.623490,1,0,5,7,5,1,1,0,1,0,6.0,0.0,132.0,19.89
1,a03223e636434f42ac4c3df47e8bac43,NaN,NaN,NaN,NaN,NaN,2017,-0.866025,-5.000000e-01,0.724793,0.688967,-0.433884,-0.900969,0,1,20,10,5,1,0,0,1,0,6.0,0.0,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,68.0,70000.0,0.0,1.0,0.0,2018,0.866025,-5.000000e-01,-0.848644,0.528964,0.433884,-0.900969,0,1,10,7,2,1,1,0,1,0,18.0,0.0,NaN,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,NaN,NaN,NaN,NaN,NaN,2017,-1.000000,-1.836970e-16,-0.937752,0.347305,0.000000,1.000000,0,1,10,10,2,1,1,1,1,0,12.0,0.0,NaN,NaN
4,68617ca6246f4fbc85e91a2a49552598,NaN,NaN,NaN,NaN,NaN,2017,-0.866025,5.000000e-01,0.394356,0.918958,0.000000,1.000000,1,0,10,5,10,1,1,1,1,0,84.0,0.0,NaN,NaN


In [7]:
# remove offers whose receivers has null values in his attributes
offer_viewed_no_null = offer_viewed[offer_viewed.isnull().iloc[:,:10].sum(axis = 1) == 0]

In [8]:
print(offer_viewed_no_null.shape)
offer_viewed_no_null.groupby("person")["offer_bogo"].count().value_counts().sort_index()

(26971, 27)


1    4328
2    5032
3    2844
4     873
5     111
Name: offer_bogo, dtype: int64

### I will make a model which predicts whether [a custmer who had offers more than 3 offers] would see the next offer or not.

In [9]:
customers_for_model = offer_viewed_no_null.groupby("person")["offer_bogo"].count().index[offer_viewed_no_null.groupby("person")["offer_bogo"].count() >= 3]
customers_for_model

Index(['0011e0d4e6b944f998e987f904e8c1e5', '0020ccbbb6d84e358d3414a3ff76cffd',
       '004c5799adbf42868b9cff0396190900', '00857b24b13f4fe0ad17b605f00357f5',
       '008d7088107b468893889da0ede0df5c', '0099bf30e4cb4265875266eb3eb25eab',
       '00ae03011f9f49b8a4b3e6d416678b0b', '00b18b535d6d4f779dea4dc9ac451478',
       '00b901d68f8f4fd68075184cd0f772d2', '00bc42a62f884b41a13cc595856cf7c3',
       ...
       'ff48bf4b65cf41a29d7086e3f742a73e', 'ff56a64e9a0e4bf5919723f31bce5965',
       'ff5d261ea11c46e4af8f85ea2cf35b61', 'ff7cc9b97c7744cfbc31bb020ea8d07e',
       'ff7fdf82996949ec82d158a02a1d9cca', 'ff8bb9517a1e4ef18b63eb9f2ba67d9e',
       'ff932c6f8bb641bd816955337d153676', 'ff992fbec53145bbae59a68d025e0458',
       'fff7576017104bcc8677a8d63322b5e1', 'ffff82501cea40309d5fdd7edcca4a07'],
      dtype='object', name='person', length=3828)

In [10]:
offer_viewed_no_null_3or_more = offer_viewed_no_null[offer_viewed_no_null.person.isin(customers_for_model)]
offer_viewed_no_null_3or_more.shape

(12579, 27)

In [11]:
offer_person_time = offer_viewed_no_null_3or_more.groupby("person").max()["t_received"]
offer_person_time.shape, offer_person_time.head()

((3828,), person
 0011e0d4e6b944f998e987f904e8c1e5    504
 0020ccbbb6d84e358d3414a3ff76cffd    504
 004c5799adbf42868b9cff0396190900    504
 00857b24b13f4fe0ad17b605f00357f5    504
 008d7088107b468893889da0ede0df5c    504
 Name: t_received, dtype: int64)

In [12]:
# Boolean mask which indicates whether the row is the last offer for the person
last_mask = (offer_viewed_no_null_3or_more["person"] + offer_viewed_no_null_3or_more["t_received"].astype(str)).isin(
        offer_person_time.index + offer_person_time.astype(str))

offer_viewed_no_null_3or_more_last = offer_viewed_no_null_3or_more[last_mask]

In [13]:
offer_viewed_no_null_3or_more_last.head()

,person,age,income,gender_F,gender_M,gender_O,became_year,became_month_sin,became_month_cos,became_day_sin,became_day_cos,became_dow_sin,became_dow_cos,offer_bogo,offer_disc,difficulty,duration,reward,email,mobile,social,web,t_received,t_viewed,amt_till_viewed,t_completed,amt_till_completed
14130,31dda685af34476cad5bc968bdb01c53,62.0,71000.0,1.0,0.0,0.0,2016,8.660254e-01,5.000000e-01,0.790776,-0.612106,0.433884,-0.900969,0,1,7,7,3,1,1,1,1,336,0.0,0.00,48.0,12.57
14136,1e9420836d554513ab90eba98552d0a9,22.0,70000.0,0.0,1.0,0.0,2017,-1.000000e+00,-1.836970e-16,-0.937752,0.347305,0.000000,1.000000,1,0,10,5,10,1,1,1,1,336,18.0,0.00,78.0,30.20
14152,99e9a1916821461ebd5d5048dc2dff7d,58.0,85000.0,1.0,0.0,0.0,2018,5.000000e-01,8.660254e-01,-0.937752,0.347305,0.433884,-0.900969,0,1,20,10,5,1,0,0,1,336,36.0,15.99,72.0,44.31
14159,9aba5058cb4f483da896335a1479a944,65.0,119000.0,1.0,0.0,0.0,2017,-1.000000e+00,-1.836970e-16,0.968077,-0.250653,-0.974928,-0.222521,1,0,5,5,5,1,1,1,1,336,18.0,0.00,42.0,29.00
14182,127cd9d95e6247e5b92bff64d6205a94,46.0,60000.0,0.0,1.0,0.0,2014,-2.449294e-16,1.000000e+00,0.299363,-0.954139,-0.781831,0.623490,0,1,10,10,2,1,1,1,1,336,30.0,3.41,48.0,15.92


# 2. Create the model.

As was written in the last notebook, the distribution of time which each customer has spent till he views the offer can be a great variable which belongs to his behavior pattern. I have to note that the distrbution should bot include the data before or the exactly that row.

In [14]:
# extract necessary columns
t_viewed = interaction.copy()[["person", "t_received", "t_viewed"]]

In [15]:
t_viewed

,person,t_received,t_viewed
0,78afa995795e4d85b5d9ceeca43f5fef,0,6.0
1,a03223e636434f42ac4c3df47e8bac43,0,6.0
2,e2127556f4f64592b11af22de27a7932,0,18.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,12.0
4,68617ca6246f4fbc85e91a2a49552598,0,84.0
5,389bc3fa690240e798340f5a15918d5c,0,0.0
6,c4863c7985cf408faee930f111475da3,0,24.0
7,aa4862eba776480b8bb9c68455b8c2e1,0,18.0
8,31dda685af34476cad5bc968bdb01c53,0,150.0
9,744d603ef08c4f33af5a61c8c7628d1c,0,48.0


In [16]:
# replace t = nan(never viewed) with 1000(arbitrary long hour : interpreted as it takes 1000 hours till he views) 
t_viewed.t_viewed = t_viewed.t_viewed.replace(np.nan,1000)

# mask of boolean which represents whether each offer was viewed until t = 120 hours
mask_over_120hour = t_viewed.t_viewed.astype(int) // 120 > 0

In [17]:
# now the offers which are not viewed until t = 120 hours are shown as 1000. 
t_viewed.loc[mask_over_120hour, "t_viewed"] = 777

t_viewed.head(10)

,person,t_received,t_viewed
0,78afa995795e4d85b5d9ceeca43f5fef,0,6.0
1,a03223e636434f42ac4c3df47e8bac43,0,6.0
2,e2127556f4f64592b11af22de27a7932,0,18.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,12.0
4,68617ca6246f4fbc85e91a2a49552598,0,84.0
5,389bc3fa690240e798340f5a15918d5c,0,0.0
6,c4863c7985cf408faee930f111475da3,0,24.0
7,aa4862eba776480b8bb9c68455b8c2e1,0,18.0
8,31dda685af34476cad5bc968bdb01c53,0,777.0
9,744d603ef08c4f33af5a61c8c7628d1c,0,48.0


In [18]:
t_viewed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48326 entries, 0 to 48325
Data columns (total 3 columns):
person        48326 non-null object
t_received    48326 non-null int64
t_viewed      48326 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ MB


In [19]:
index = pd.Series(t_viewed.t_viewed.unique()).sort_values()
df = pd.DataFrame(columns=index)
df["person"] = offer_viewed_no_null_3or_more_last.person
df = df.replace(np.nan, 0)
df.head()

,0.0,6.0,12.0,18.0,24.0,30.0,36.0,42.0,48.0,54.0,60.0,66.0,72.0,78.0,84.0,90.0,96.0,102.0,108.0,114.0,777.0,person
14130,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,31dda685af34476cad5bc968bdb01c53
14136,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1e9420836d554513ab90eba98552d0a9
14152,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,99e9a1916821461ebd5d5048dc2dff7d
14159,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9aba5058cb4f483da896335a1479a944
14182,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,127cd9d95e6247e5b92bff64d6205a94


In [54]:
t = time.time()

person_index = []

# for each of the row which represents the last offer to each person who has got >= 3 offers...
for i, (each_person, each_time) in enumerate(zip(offer_viewed_no_null_3or_more_last.person, offer_viewed_no_null_3or_more_last.t_received)):
    
    # offers sent to that person BEFORE that last offer
    offers_before =  t_viewed[(t_viewed.person == each_person) & (t_viewed.t_received < each_time)]
    
    for t in offers_before.t_viewed:
        temp_row = df.iloc[i].copy()
        temp_row.loc[t] += 1
        df.iloc[i] = temp_row

time.time() - t

1554831734.400788

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3828 entries, 14130 to 32006
Data columns (total 22 columns):
0.0       3828 non-null int64
6.0       3828 non-null int64
12.0      3828 non-null int64
18.0      3828 non-null int64
24.0      3828 non-null int64
30.0      3828 non-null int64
36.0      3828 non-null int64
42.0      3828 non-null int64
48.0      3828 non-null int64
54.0      3828 non-null int64
60.0      3828 non-null int64
66.0      3828 non-null int64
72.0      3828 non-null int64
78.0      3828 non-null int64
84.0      3828 non-null int64
90.0      3828 non-null int64
96.0      3828 non-null int64
102.0     3828 non-null int64
108.0     3828 non-null int64
114.0     3828 non-null int64
777.0     3828 non-null int64
person    3828 non-null object
dtypes: int64(21), object(1)
memory usage: 687.8+ KB


In [56]:
df2 = df.replace(np.nan, 0)
df2.iloc[:,:-1] = df2.iloc[:,:-1].cumsum(axis=1)
df2 = df2.rename(columns = {777:"denom_viewed"})
df2.head()

,0.0,6.0,12.0,18.0,24.0,30.0,36.0,42.0,48.0,54.0,60.0,66.0,72.0,78.0,84.0,90.0,96.0,102.0,108.0,114.0,denom_viewed,person
14130,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,31dda685af34476cad5bc968bdb01c53
14136,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,1e9420836d554513ab90eba98552d0a9
14152,0,0,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,99e9a1916821461ebd5d5048dc2dff7d
14159,0,0,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,9aba5058cb4f483da896335a1479a944
14182,0,0,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,127cd9d95e6247e5b92bff64d6205a94


In [57]:
for column_name in df2.columns[:-2]:
        df2[column_name] = df2[column_name].astype(float) / df2.denom_viewed

In [58]:
df2.head()
df2.to_csv("data/hour_vs_viewed_modified", index=False)

In [59]:
interaction[interaction.person ==
        "1e9420836d554513ab90eba98552d0a9"]

,person,age,income,gender_F,gender_M,gender_O,became_year,became_month_sin,became_month_cos,became_day_sin,became_day_cos,became_dow_sin,became_dow_cos,offer_bogo,offer_disc,difficulty,duration,reward,email,mobile,social,web,t_received,t_viewed,amt_till_viewed,t_completed,amt_till_completed
17,1e9420836d554513ab90eba98552d0a9,22.0,70000.0,0.0,1.0,0.0,2017,-1.0,-1.836970e-16,-0.937752,0.347305,0.0,1.0,1,0,10,7,10,1,1,1,0,0,96.0,0.00,NaN,NaN
10129,1e9420836d554513ab90eba98552d0a9,22.0,70000.0,0.0,1.0,0.0,2017,-1.0,-1.836970e-16,-0.937752,0.347305,0.0,1.0,1,0,10,5,10,1,1,1,1,168,6.0,0.00,30.0,17.11
20258,1e9420836d554513ab90eba98552d0a9,22.0,70000.0,0.0,1.0,0.0,2017,-1.0,-1.836970e-16,-0.937752,0.347305,0.0,1.0,1,0,10,5,10,1,1,1,1,336,18.0,0.00,78.0,30.20
30439,1e9420836d554513ab90eba98552d0a9,22.0,70000.0,0.0,1.0,0.0,2017,-1.0,-1.836970e-16,-0.937752,0.347305,0.0,1.0,0,1,7,7,3,1,1,1,1,408,6.0,20.63,6.0,20.63


In [60]:
merged_df = offer_viewed_no_null_3or_more_last.merge(df2)
merged_df.head()

,person,age,income,gender_F,gender_M,gender_O,became_year,became_month_sin,became_month_cos,became_day_sin,became_day_cos,became_dow_sin,became_dow_cos,offer_bogo,offer_disc,difficulty,duration,reward,email,mobile,social,web,t_received,t_viewed,amt_till_viewed,t_completed,amt_till_completed,0.0,6.0,12.0,18.0,24.0,30.0,36.0,42.0,48.0,54.0,60.0,66.0,72.0,78.0,84.0,90.0,96.0,102.0,108.0,114.0,denom_viewed
0,31dda685af34476cad5bc968bdb01c53,62.0,71000.0,1.0,0.0,0.0,2016,8.660254e-01,5.000000e-01,0.790776,-0.612106,0.433884,-0.900969,0,1,7,7,3,1,1,1,1,336,0.0,0.00,48.0,12.57,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,2
1,1e9420836d554513ab90eba98552d0a9,22.0,70000.0,0.0,1.0,0.0,2017,-1.000000e+00,-1.836970e-16,-0.937752,0.347305,0.000000,1.000000,1,0,10,5,10,1,1,1,1,336,18.0,0.00,78.0,30.20,0.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0,2
2,99e9a1916821461ebd5d5048dc2dff7d,58.0,85000.0,1.0,0.0,0.0,2018,5.000000e-01,8.660254e-01,-0.937752,0.347305,0.433884,-0.900969,0,1,20,10,5,1,0,0,1,336,36.0,15.99,72.0,44.31,0.0,0.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2
3,9aba5058cb4f483da896335a1479a944,65.0,119000.0,1.0,0.0,0.0,2017,-1.000000e+00,-1.836970e-16,0.968077,-0.250653,-0.974928,-0.222521,1,0,5,5,5,1,1,1,1,336,18.0,0.00,42.0,29.00,0.0,0.0,0.5,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2
4,127cd9d95e6247e5b92bff64d6205a94,46.0,60000.0,0.0,1.0,0.0,2014,-2.449294e-16,1.000000e+00,0.299363,-0.954139,-0.781831,0.623490,0,1,10,10,2,1,1,1,1,336,30.0,3.41,48.0,15.92,0.0,0.0,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2


In [61]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3828 entries, 14130 to 32006
Data columns (total 22 columns):
0.0             3828 non-null float64
6.0             3828 non-null float64
12.0            3828 non-null float64
18.0            3828 non-null float64
24.0            3828 non-null float64
30.0            3828 non-null float64
36.0            3828 non-null float64
42.0            3828 non-null float64
48.0            3828 non-null float64
54.0            3828 non-null float64
60.0            3828 non-null float64
66.0            3828 non-null float64
72.0            3828 non-null float64
78.0            3828 non-null float64
84.0            3828 non-null float64
90.0            3828 non-null float64
96.0            3828 non-null float64
102.0           3828 non-null float64
108.0           3828 non-null float64
114.0           3828 non-null float64
denom_viewed    3828 non-null int64
person          3828 non-null object
dtypes: float64(20), int64(1), object(1)
memory usage: 

In [62]:
merged_df.columns

Index([            'person',                'age',             'income',
                 'gender_F',           'gender_M',           'gender_O',
              'became_year',   'became_month_sin',   'became_month_cos',
           'became_day_sin',     'became_day_cos',     'became_dow_sin',
           'became_dow_cos',         'offer_bogo',         'offer_disc',
               'difficulty',           'duration',             'reward',
                    'email',             'mobile',             'social',
                      'web',         't_received',           't_viewed',
          'amt_till_viewed',        't_completed', 'amt_till_completed',
                        0.0,                  6.0,                 12.0,
                       18.0,                 24.0,                 30.0,
                       36.0,                 42.0,                 48.0,
                       54.0,                 60.0,                 66.0,
                       72.0,                 78.0, 

In [63]:
merged_df

,person,age,income,gender_F,gender_M,gender_O,became_year,became_month_sin,became_month_cos,became_day_sin,became_day_cos,became_dow_sin,became_dow_cos,offer_bogo,offer_disc,difficulty,duration,reward,email,mobile,social,web,t_received,t_viewed,amt_till_viewed,t_completed,amt_till_completed,0.0,6.0,12.0,18.0,24.0,30.0,36.0,42.0,48.0,54.0,60.0,66.0,72.0,78.0,84.0,90.0,96.0,102.0,108.0,114.0,denom_viewed
0,31dda685af34476cad5bc968bdb01c53,62.0,71000.0,1.0,0.0,0.0,2016,8.660254e-01,5.000000e-01,7.907757e-01,-0.612106,0.433884,-0.900969,0,1,7,7,3,1,1,1,1,336,0.0,0.00,48.0,12.57,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,2
1,1e9420836d554513ab90eba98552d0a9,22.0,70000.0,0.0,1.0,0.0,2017,-1.000000e+00,-1.836970e-16,-9.377521e-01,0.347305,0.000000,1.000000,1,0,10,5,10,1,1,1,1,336,18.0,0.00,78.0,30.20,0.000000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,1.000000,1.000000,1.000000,1.000000,2
2,99e9a1916821461ebd5d5048dc2dff7d,58.0,85000.0,1.0,0.0,0.0,2018,5.000000e-01,8.660254e-01,-9.377521e-01,0.347305,0.433884,-0.900969,0,1,20,10,5,1,0,0,1,336,36.0,15.99,72.0,44.31,0.000000,0.000000,0.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2
3,9aba5058cb4f483da896335a1479a944,65.0,119000.0,1.0,0.0,0.0,2017,-1.000000e+00,-1.836970e-16,9.680771e-01,-0.250653,-0.974928,-0.222521,1,0,5,5,5,1,1,1,1,336,18.0,0.00,42.0,29.00,0.000000,0.000000,0.500000,0.500000,0.500000,0.500000,0.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2
4,127cd9d95e6247e5b92bff64d6205a94,46.0,60000.0,0.0,1.0,0.0,2014,-2.449294e-16,1.000000e+00,2.993631e-01,-0.954139,-0.781831,0.623490,0,1,10,10,2,1,1,1,1,336,30.0,3.41,48.0,15.92,0.000000,0.000000,0.500000,0.500000,0.500000,0.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2
5,031387fa9cfd47e88f4e4b4b4cb71925,41.0,83000.0,0.0,1.0,0.0,2017,-1.000000e+00,-1.836970e-16,-9.680771e-01,-0.250653,-0.433884,-0.900969,0,1,20,10,5,1,0,0,1,336,12.0,0.00,NaN,NaN,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2
6,32fbf97e89274e8a935903f2ad095e1f,73.0,38000.0,0.0,1.0,0.0,2017,-2.449294e-16,1.000000e+00,6.513725e-01,-0.758758,0.781831,0.623490,1,0,5,7,5,1,1,0,1,336,24.0,0.00,30.0,11.73,0.000000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,1.000000,1.000000,1.000000,1.000000,2
7,371d6b3a983547709252bbbcbedda147,58.0,76000.0,1.0,0.0,0.0,2017,8.660254e-01,5.000000e-01,-6.513725e-01,-0.758758,-0.781831,0.623490,0,1,7,7,3,1,1,1,1,336,36.0,0.00,114.0,31.76,0.000000,0.500000,0.500000,0.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2
8,39069e7cd52e474d8c97d8162609bca1,71.0,51000.0,0.0,1.0,0.0,2017,-2.449294e-16,1.000000e+00,7.247928e-01,0.688967,0.000000,1.000000,1,0,10,5,10,1,1,1,1,336,48.0,0.00,108.0,16.22,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2
9,0e72fad934a74c2c97a98c525d64f738,59.0,59000.0,1.0,0.0,0.0,2018,5.000000e-01,8.660254e-01,-1.011683e-01,-0.994869,0.781831,0.623490,0,1,7,7,3,1,1,1,1,336,12.0,0.00,30.0,10.45,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0000

In [64]:
# create the y label 
y_completed = ~(merged_df.t_completed.isnull())
y_completed = y_completed.astype(int)
y_completed.head()

0    1
1    1
2    1
3    1
4    1
Name: t_completed, dtype: int64

In [65]:
indep_var = [         'age',             'income',
                 'gender_F',           'gender_M',           'gender_O',
              'became_year',   'became_month_sin',   'became_month_cos',
           'became_day_sin',     'became_day_cos',     'became_dow_sin',
           'became_dow_cos',         'offer_bogo',         'offer_disc',
               'difficulty',           'duration',             'reward',
                    'email',             'mobile',             'social',
                      'web',         't_received',      
                        0.0,                  6.0,                 12.0,
                       18.0,                 24.0,                 30.0,
                       36.0,                 42.0,                 48.0,
                       54.0,                 60.0,                 66.0,
                       72.0,                 78.0,                 84.0,
                       90.0,                 96.0,                102.0,
                      108.0,                114.0,       'denom_viewed']
indep_var

['age',
 'income',
 'gender_F',
 'gender_M',
 'gender_O',
 'became_year',
 'became_month_sin',
 'became_month_cos',
 'became_day_sin',
 'became_day_cos',
 'became_dow_sin',
 'became_dow_cos',
 'offer_bogo',
 'offer_disc',
 'difficulty',
 'duration',
 'reward',
 'email',
 'mobile',
 'social',
 'web',
 't_received',
 0.0,
 6.0,
 12.0,
 18.0,
 24.0,
 30.0,
 36.0,
 42.0,
 48.0,
 54.0,
 60.0,
 66.0,
 72.0,
 78.0,
 84.0,
 90.0,
 96.0,
 102.0,
 108.0,
 114.0,
 'denom_viewed']

In [66]:
# remove offers whose receivers has null values in his attributes
offer_viewed_no_null = merged_df[merged_df.isnull().iloc[:,:25].sum(axis = 1) == 0]
y_completed_no_null = y_completed[offer_viewed.isnull().iloc[:,:25].sum(axis = 1) == 0]
y_completed_no_null.shape

(3246,)

In [67]:
merged_df

,person,age,income,gender_F,gender_M,gender_O,became_year,became_month_sin,became_month_cos,became_day_sin,became_day_cos,became_dow_sin,became_dow_cos,offer_bogo,offer_disc,difficulty,duration,reward,email,mobile,social,web,t_received,t_viewed,amt_till_viewed,t_completed,amt_till_completed,0.0,6.0,12.0,18.0,24.0,30.0,36.0,42.0,48.0,54.0,60.0,66.0,72.0,78.0,84.0,90.0,96.0,102.0,108.0,114.0,denom_viewed
0,31dda685af34476cad5bc968bdb01c53,62.0,71000.0,1.0,0.0,0.0,2016,8.660254e-01,5.000000e-01,7.907757e-01,-0.612106,0.433884,-0.900969,0,1,7,7,3,1,1,1,1,336,0.0,0.00,48.0,12.57,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,2
1,1e9420836d554513ab90eba98552d0a9,22.0,70000.0,0.0,1.0,0.0,2017,-1.000000e+00,-1.836970e-16,-9.377521e-01,0.347305,0.000000,1.000000,1,0,10,5,10,1,1,1,1,336,18.0,0.00,78.0,30.20,0.000000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,1.000000,1.000000,1.000000,1.000000,2
2,99e9a1916821461ebd5d5048dc2dff7d,58.0,85000.0,1.0,0.0,0.0,2018,5.000000e-01,8.660254e-01,-9.377521e-01,0.347305,0.433884,-0.900969,0,1,20,10,5,1,0,0,1,336,36.0,15.99,72.0,44.31,0.000000,0.000000,0.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2
3,9aba5058cb4f483da896335a1479a944,65.0,119000.0,1.0,0.0,0.0,2017,-1.000000e+00,-1.836970e-16,9.680771e-01,-0.250653,-0.974928,-0.222521,1,0,5,5,5,1,1,1,1,336,18.0,0.00,42.0,29.00,0.000000,0.000000,0.500000,0.500000,0.500000,0.500000,0.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2
4,127cd9d95e6247e5b92bff64d6205a94,46.0,60000.0,0.0,1.0,0.0,2014,-2.449294e-16,1.000000e+00,2.993631e-01,-0.954139,-0.781831,0.623490,0,1,10,10,2,1,1,1,1,336,30.0,3.41,48.0,15.92,0.000000,0.000000,0.500000,0.500000,0.500000,0.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2
5,031387fa9cfd47e88f4e4b4b4cb71925,41.0,83000.0,0.0,1.0,0.0,2017,-1.000000e+00,-1.836970e-16,-9.680771e-01,-0.250653,-0.433884,-0.900969,0,1,20,10,5,1,0,0,1,336,12.0,0.00,NaN,NaN,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2
6,32fbf97e89274e8a935903f2ad095e1f,73.0,38000.0,0.0,1.0,0.0,2017,-2.449294e-16,1.000000e+00,6.513725e-01,-0.758758,0.781831,0.623490,1,0,5,7,5,1,1,0,1,336,24.0,0.00,30.0,11.73,0.000000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,1.000000,1.000000,1.000000,1.000000,2
7,371d6b3a983547709252bbbcbedda147,58.0,76000.0,1.0,0.0,0.0,2017,8.660254e-01,5.000000e-01,-6.513725e-01,-0.758758,-0.781831,0.623490,0,1,7,7,3,1,1,1,1,336,36.0,0.00,114.0,31.76,0.000000,0.500000,0.500000,0.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2
8,39069e7cd52e474d8c97d8162609bca1,71.0,51000.0,0.0,1.0,0.0,2017,-2.449294e-16,1.000000e+00,7.247928e-01,0.688967,0.000000,1.000000,1,0,10,5,10,1,1,1,1,336,48.0,0.00,108.0,16.22,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2
9,0e72fad934a74c2c97a98c525d64f738,59.0,59000.0,1.0,0.0,0.0,2018,5.000000e-01,8.660254e-01,-1.011683e-01,-0.994869,0.781831,0.623490,0,1,7,7,3,1,1,1,1,336,12.0,0.00,30.0,10.45,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0000

In [79]:
# create a column which represents the probability of the past offers accomplished.

dict_person_to_past_rate = {}

for each_person, each_time in zip(merged_df.person, merged_df.t_received):
    prev_trans_df = offer_viewed_no_null_3or_more[(offer_viewed_no_null_3or_more.person == each_person) & (offer_viewed_no_null_3or_more.t_received < each_time)]
    print(prev_trans_df.t_completed.count() / prev_trans_df.shape[0])

0.5
0.5
1.0
1.0
0.5
0.0
1.0
1.0
0.0
1.0
1.0
1.0
0.5
0.0
0.0
1.0
0.0
1.0
1.0
0.5
0.0
1.0
1.0
0.5
0.0
0.0
1.0
0.0
0.0
0.5
1.0
0.5
1.0
0.0
0.5
1.0
0.0
0.5
0.0
1.0
0.5
1.0
1.0
1.0
1.0
0.0
0.5
0.0
0.5
0.0
1.0
1.0
1.0
0.0
0.0
0.0
1.0
0.0
0.5
0.5
1.0
1.0
0.5
1.0
1.0
0.5
0.0
1.0
1.0
0.0
1.0
0.5
1.0
0.5
1.0
1.0
1.0
0.0
0.5
0.5
0.5
1.0
1.0
1.0
0.5
1.0
0.0
0.0
1.0
0.5
0.0
1.0
0.5
0.0
0.0
0.0
0.0
1.0
1.0
1.0
0.0
1.0
0.0
0.0
0.5
1.0
0.5
1.0
1.0
1.0
1.0
0.0
0.0
1.0
0.5
0.5
1.0
0.0
0.5
1.0
1.0
0.5
0.0
1.0
0.0
0.0
0.5
1.0
0.0
0.0
0.0
0.5
1.0
0.5
1.0
1.0
0.0
1.0
0.5
1.0
1.0
1.0
0.5
1.0
0.5
0.5
0.5
1.0
1.0
1.0
1.0
1.0
0.5
1.0
0.5
1.0
1.0
1.0
0.5
0.5
0.5
1.0
0.5
0.5
1.0
1.0
0.0
1.0
0.0
1.0
0.0
1.0
1.0
1.0
0.5
0.0
0.5
1.0
0.5
0.5
0.0
1.0
1.0
1.0
0.5
1.0
1.0
0.0
1.0
1.0
0.0
1.0
0.5
0.5
1.0
0.0
1.0
0.5
1.0
1.0
1.0
1.0
0.0
0.5
0.5
0.5
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.5
0.5
1.0
0.5
1.0
0.0
0.5
0.5
0.0
1.0
0.5
1.0
0.5
1.0
0.5
0.5
1.0
0.0
1.0
0.5
0.5
0.0
1.0
0.5
1.0
0.5
1.0
1.0
1.0
0.5
0.0
1.0
0.0
0.0
1.0
0.5
0.0


0.5
1.0
0.0
1.0
0.5
1.0
1.0
1.0
0.6666666666666666
0.0
0.5
1.0
0.0
1.0
1.0
0.5
1.0
0.5
0.0
1.0
1.0
0.5
1.0
0.0
1.0
1.0
0.5
0.5
0.3333333333333333
1.0
1.0
0.5
0.5
1.0
0.75
0.0
0.6666666666666666
0.5
0.5
0.5
1.0
0.0
1.0
0.25
0.3333333333333333
0.0
0.6666666666666666
0.6666666666666666
1.0
0.0
1.0
0.0
0.3333333333333333
1.0
1.0
1.0
0.6666666666666666
0.0
0.6666666666666666
1.0
1.0
0.0
1.0
0.6666666666666666
0.0
1.0
0.5
1.0
1.0
0.3333333333333333
0.0
0.5
0.0
0.5
0.0
0.6666666666666666
0.6666666666666666
1.0
1.0
0.0
0.0
0.0
0.5
1.0
0.3333333333333333
1.0
0.0
0.0
0.0
1.0
1.0
0.6666666666666666
1.0
0.3333333333333333
1.0
1.0
0.5
1.0
1.0
1.0
1.0
1.0
0.6666666666666666
1.0
1.0
1.0
0.0
0.0
0.0
0.5
1.0
1.0
0.6666666666666666
0.3333333333333333
1.0
0.0
0.0
1.0
1.0
0.5
0.5
0.0
1.0
0.5
0.3333333333333333
1.0
1.0
0.0
1.0
0.5
0.6666666666666666
1.0
1.0
0.6666666666666666
0.3333333333333333
0.5
0.5
1.0
1.0
0.5
0.5
1.0
1.0
0.0
1.0
0.0
1.0
1.0
1.0
0.3333333333333333
0.5
0.5
0.0
1.0
1.0
0.5
0.5
1.0
1.0
1.

1.0
0.5
0.5
1.0
1.0
0.0
0.0
0.0
1.0
1.0
0.5
1.0
0.5
0.5
0.5
0.0
0.0
0.0
0.5
0.0
1.0
1.0
1.0
1.0
0.0
0.5
1.0
1.0
1.0
1.0
1.0
1.0
0.0
0.3333333333333333
1.0
1.0
0.5
0.0
0.5
0.5
0.5
1.0
0.75
1.0
1.0
0.0
1.0
0.75
0.5
1.0
0.3333333333333333
0.5
0.6666666666666666
1.0
1.0
0.6666666666666666
1.0
0.0
0.0
0.5
1.0
1.0
0.5
1.0
1.0
1.0
1.0
1.0
1.0
0.6666666666666666
0.3333333333333333
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
0.5
0.5
1.0
1.0
0.5
1.0
1.0
0.5
1.0
1.0
1.0
0.3333333333333333
1.0
0.0
0.5
0.0
0.0
1.0
1.0
1.0
1.0
1.0
0.5
0.0
0.6666666666666666
0.6666666666666666
1.0
1.0
0.5
0.5
1.0
1.0
1.0
0.3333333333333333
0.5
1.0
0.5
0.0
0.0
0.5
0.5
0.0
0.5
0.5
1.0
0.0
0.0
0.5
0.5
0.5
1.0
0.5
1.0
1.0
1.0
1.0
0.5
1.0
0.5
0.75
0.5
0.5
1.0
0.5
0.0
0.0
0.25
0.6666666666666666
0.5
0.75
0.0
0.0
0.5
1.0
1.0
0.3333333333333333
0.0
1.0
0.5
0.5
1.0
0.5
1.0
1.0
0.3333333333333333
0.5
1.0
1.0
1.0
0.0
1.0
0.5
1.0
1.0
0.5
0.5
1.0
0.5
0.6666666666666666
0.0
0.5
0.75
0.0
0.0
1.0
0.3333333333333333
1.0
0.0
1.0
1.0
1.0
1.0
1.0
0

In [73]:
offer_viewed_no_null_3or_more.head()

,person,age,income,gender_F,gender_M,gender_O,became_year,became_month_sin,became_month_cos,became_day_sin,became_day_cos,became_dow_sin,became_dow_cos,offer_bogo,offer_disc,difficulty,duration,reward,email,mobile,social,web,t_received,t_viewed,amt_till_viewed,t_completed,amt_till_completed
5,389bc3fa690240e798340f5a15918d5c,65.0,53000.0,0.0,1.0,0.0,2018,0.866025,5.000000e-01,0.968077,-0.250653,-0.433884,-0.900969,1,0,5,5,5,1,1,1,1,0,0.0,0.0,60.0,9.54
8,31dda685af34476cad5bc968bdb01c53,62.0,71000.0,1.0,0.0,0.0,2016,0.866025,5.000000e-01,0.790776,-0.612106,0.433884,-0.900969,0,1,20,10,5,1,0,0,1,0,150.0,0.0,NaN,NaN
12,f806632c011441378d4646567f357a21,42.0,69000.0,0.0,1.0,0.0,2013,-0.866025,-5.000000e-01,0.790776,-0.612106,-0.781831,0.623490,0,1,10,10,2,1,1,1,1,0,18.0,0.0,126.0,28.29
13,d058f73bf8674a26a95227db098147b1,56.0,88000.0,1.0,0.0,0.0,2018,0.866025,-5.000000e-01,-0.571268,0.820763,-0.974928,-0.222521,0,1,20,10,5,1,0,0,1,0,30.0,0.0,192.0,47.14
14,1e9420836d554513ab90eba98552d0a9,22.0,70000.0,0.0,1.0,0.0,2017,-1.000000,-1.836970e-16,-0.937752,0.347305,0.000000,1.000000,1,0,10,7,10,1,1,1,0,0,96.0,0.0,NaN,NaN


In [68]:
# proportion of 0(viewed and completed) vs 1(viewed but not comlpeted)
merged_df.t_completed.count() / merged_df.shape[0]

0.591692789968652

# How many datas do I have?

### Split them into training and testing data

https://scikit-learn.org/stable/modules/cross_validation.html

In [69]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(merged_df[indep_var], y_completed, test_size=0.10)

In [70]:
y_train.mean(), y_test.mean()

(0.5912917271407837, 0.5953002610966057)

# Model using attribute, distribution of time till viewing, completement rate of past offers.

## 1. LogisticRegression


In [71]:
X_train

,age,income,gender_F,gender_M,gender_O,became_year,became_month_sin,became_month_cos,became_day_sin,became_day_cos,became_dow_sin,became_dow_cos,offer_bogo,offer_disc,difficulty,duration,reward,email,mobile,social,web,t_received,0.0,6.0,12.0,18.0,24.0,30.0,36.0,42.0,48.0,54.0,60.0,66.0,72.0,78.0,84.0,90.0,96.0,102.0,108.0,114.0,denom_viewed
1276,81.0,40000.0,0.0,1.0,0.0,2017,-8.660254e-01,5.000000e-01,-9.377521e-01,0.347305,0.974928,-0.222521,1,0,5,5,5,1,1,1,1,408,0.500000,0.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2
2698,69.0,86000.0,0.0,1.0,0.0,2016,-5.000000e-01,8.660254e-01,9.884683e-01,0.151428,0.000000,1.000000,1,0,5,7,5,1,1,0,1,504,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,1.000000,1.000000,2
1504,78.0,58000.0,0.0,1.0,0.0,2018,8.660254e-01,-5.000000e-01,-8.978045e-01,-0.440394,-0.974928,-0.222521,1,0,10,7,10,1,1,1,0,408,0.000000,0.000000,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,3
1981,62.0,70000.0,1.0,0.0,0.0,2018,8.660254e-01,5.000000e-01,-1.011683e-01,-0.994869,-0.433884,-0.900969,1,0,10,5,10,1,1,1,1,504,0.500000,0.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2
767,26.0,40000.0,0.0,1.0,0.0,2018,5.000000e-01,8.660254e-01,6.513725e-01,-0.758758,-0.433884,-0.900969,0,1,7,7,3,1,1,1,1,408,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.666667,0.666667,0.666667,3
3475,62.0,111000.0,0.0,1.0,0.0,2016,-8.660254e-01,-5.000000e-01,5.712682e-01,0.820763,0.974928,-0.222521,1,0,10,7,10,1,1,1,0,504,0.333333,0.333333,0.333333,0.666667,0.666667,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3
140,70.0,31000.0,1.0,0.0,0.0,2014,5.000000e-01,8.660254e-01,-2.012985e-01,0.979530,0.433884,-0.900969,1,0,10,7,10,1,1,1,0,336,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2
1767,73.0,46000.0,0.0,1.0,0.0,2018,1.224647e-16,-1.000000e+00,8.978045e-01,-0.440394,-0.781831,0.623490,1,0,10,5,10,1,1,1,1,504,0.333333,0.333333,0.333333,0.333333,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,3
2606,53.0,67000.0,1.0,0.0,0.0,2016,-2.449294e-16,1.000000e+00,2.012985e-01,0.979530,0.433884,-0.900969,1,0,5,5,5,1,1,1,1,504,0.666667,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3
1431,75.0,57000.0,1.0,0.0,0.0,2018,8.660254e-01,5.000000e-01,-7.247928e-01,0.688967,0.781831,0.623490,1,0,5,5,5,1,1,1,1,408,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2


In [40]:
from sklearn import linear_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [41]:
mlr = linear_model.LogisticRegression()
mlr.fit(X=X_train[indep_var], y=y_train)


y_train_pred = mlr.predict(X_train[indep_var])
y_test_pred = mlr.predict(X_test[indep_var])

# evaluation of the model by trining data
print("accuracy : {}".format(accuracy_score(y_train, y_train_pred)))
print("percision: {}".format(precision_score(y_train, y_train_pred)))
print("recall   : {}".format(recall_score(y_train, y_train_pred)))
print("f1       : {}\n".format(f1_score(y_train, y_train_pred)))

# evaluation of the model by trainig data
print("accuracy : {}".format(accuracy_score(y_test,y_test_pred)))
print("percision: {}".format(precision_score(y_test,y_test_pred)))
print("recall   : {}".format(recall_score(y_test,y_test_pred)))
print("f1       : {}".format(f1_score(y_test,y_test_pred)))

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

## 3. RandomForestClassifier

## from sklearn.ensemble import RandomForestClassifier

In [496]:
rfc = RandomForestClassifier(min_samples_leaf=100)
rfc.fit(X=X_train[indep_var], y=y_train)

y_train_pred = rfc.predict(X_train[indep_var])
y_test_pred = rfc.predict(X_test[indep_var])

# evaluation of the model by trining data
print("accuracy : {}".format(accuracy_score(y_train, y_train_pred)))
print("percision: {}".format(precision_score(y_train, y_train_pred)))
print("recall   : {}".format(recall_score(y_train, y_train_pred)))
print("f1       : {}\n".format(f1_score(y_train, y_train_pred)))

# evaluation of the model by trainig data
print("accuracy : {}".format(accuracy_score(y_test,y_test_pred)))
print("percision: {}".format(precision_score(y_test,y_test_pred)))
print("recall   : {}".format(recall_score(y_test,y_test_pred)))
print("f1       : {}".format(f1_score(y_test,y_test_pred)))

accuracy : 0.6925979680696662
percision: 0.6901244167962675
recall   : 0.8713794796269023
f1       : 0.7702321544803645

accuracy : 0.6631853785900783
percision: 0.6761565836298933
recall   : 0.8333333333333334
f1       : 0.7465618860510806


/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


# 4. Neural Network Classifier

In [487]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(1000,1000,200), random_state=1)
mlp.fit(X_train[indep_var], y_train)

y_train_pred = mlp.predict(X_train[indep_var])
y_test_pred = mlp.predict(X_test[indep_var])

# evaluation of the model by trining data
print("accuracy : {}".format(accuracy_score(y_train, y_train_pred)))
print("percision: {}".format(precision_score(y_train, y_train_pred)))
print("recall   : {}".format(recall_score(y_train, y_train_pred)))
print("f1       : {}\n".format(f1_score(y_train, y_train_pred)))

# evaluation of the model by trainig data
print("accuracy : {}".format(accuracy_score(y_test,y_test_pred)))
print("percision: {}".format(precision_score(y_test,y_test_pred)))
print("recall   : {}".format(recall_score(y_test,y_test_pred)))
print("f1       : {}".format(f1_score(y_test,y_test_pred)))

accuracy : 0.5912917271407837
percision: 0.5912917271407837
recall   : 1.0
f1       : 0.743159430864648

accuracy : 0.5953002610966057
percision: 0.5953002610966057
recall   : 1.0
f1       : 0.7463175122749591


In [499]:
indep_var2 = [         'age',             'income',
                 'gender_F',           'gender_M',           'gender_O',
              'became_year',         'offer_bogo',         'offer_disc',
               'difficulty',           'duration',         't_received',      
                        0.0,                  6.0,                 12.0,
                       18.0,                 24.0,                 30.0,
                       36.0,                 42.0,                 48.0,
                       54.0,                 60.0,                 66.0,
                       72.0,                 78.0,                 84.0,
                       90.0,                 96.0,                102.0,
                      108.0,                114.0,       'denom_viewed']
indep_var

['age',
 'income',
 'gender_F',
 'gender_M',
 'gender_O',
 'became_year',
 'became_month_sin',
 'became_month_cos',
 'became_day_sin',
 'became_day_cos',
 'became_dow_sin',
 'became_dow_cos',
 'offer_bogo',
 'offer_disc',
 'difficulty',
 'duration',
 'reward',
 'email',
 'mobile',
 'social',
 'web',
 't_received',
 0.0,
 6.0,
 12.0,
 18.0,
 24.0,
 30.0,
 36.0,
 42.0,
 48.0,
 54.0,
 60.0,
 66.0,
 72.0,
 78.0,
 84.0,
 90.0,
 96.0,
 102.0,
 108.0,
 114.0,
 'denom_viewed']

In [501]:
rfc = RandomForestClassifier(min_samples_leaf=100)
rfc.fit(X=X_train[indep_var2], y=y_train)

y_train_pred = rfc.predict(X_train[indep_var2])
y_test_pred = rfc.predict(X_test[indep_var2])

# evaluation of the model by trining data
print("accuracy : {}".format(accuracy_score(y_train, y_train_pred)))
print("percision: {}".format(precision_score(y_train, y_train_pred)))
print("recall   : {}".format(recall_score(y_train, y_train_pred)))
print("f1       : {}\n".format(f1_score(y_train, y_train_pred)))

# evaluation of the model by trainig data
print("accuracy : {}".format(accuracy_score(y_test,y_test_pred)))
print("percision: {}".format(precision_score(y_test,y_test_pred)))
print("recall   : {}".format(recall_score(y_test,y_test_pred)))
print("f1       : {}".format(f1_score(y_test,y_test_pred)))

accuracy : 0.7021770682148041
percision: 0.7005156683855612
recall   : 0.8669612174766814
f1       : 0.7749012724879332

accuracy : 0.6762402088772846
percision: 0.6857142857142857
recall   : 0.8421052631578947
f1       : 0.7559055118110235


/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [493]:
transcript.sort_values(by=["person","time"])

,event,person,time,transaction,id
55972,offer received,0009655768c64bdeb2e877511632db8f,168,NaN,info_3days
77705,offer viewed,0009655768c64bdeb2e877511632db8f,192,NaN,info_3days
89291,transaction,0009655768c64bdeb2e877511632db8f,228,22.16,NaN
113605,offer received,0009655768c64bdeb2e877511632db8f,336,NaN,info_4days
139992,offer viewed,0009655768c64bdeb2e877511632db8f,372,NaN,info_4days
153401,offer received,0009655768c64bdeb2e877511632db8f,408,NaN,bogo_5days$5
168412,transaction,0009655768c64bdeb2e877511632db8f,414,8.57,NaN
168413,offer completed,0009655768c64bdeb2e877511632db8f,414,NaN,bogo_5days$5
187554,offer viewed,0009655768c64bdeb2e877511632db8f,456,NaN,bogo_5days$5
204340,offer received,0009655768c64bdeb2e877511632db8f,504,NaN,disc_10days$10-$2


In [498]:
transcript.sort_values("people")

KeyError: 'people'

# So how to use each person's past behavior??

In [ ]:
# offers which are viewed before completed or viewed but never completed. 
## (which are sent to customers with full attributes)
offer_viewed_no_null.groupby("person").count()["t_viewed"].value_counts()

Based on this data I determined to make a model which predicts predicts the completion rate 

* predict 3rd offer interaction from past 2
* predict 4th offer interaction from past 3
* predict 5th offer interaction from past 4

In [ ]:
offer_viewed_no_null.sort_values(by="person")

### First Attribute : how many hours a user view the offer.
# NOTE : This cannot be used I made a mistake to include the transactions which would be in the test data later. So file "hour vs viewed.csv" should not be used.

### Also I will create new notebook "190403StarbucksPromotionAnalysis.ipynb" and cotinue analysis there.

In this section, I decided to count what persentage of offer has been viewed every 6 hours in 5 days = 120 hours.
I ignore the data of offers which were seen completed first and then viewed (or both at the same time) / completed first but never viewed.
(To measure the cases when a customer notices and views the offer on the effect of completment)

As a premise, I assume that 

* View rate does not depend on the the channel of the offer.
* The tendency to notice an offer after completing and the tendency of viewing an offer are independent.

In [ ]:
#offer_not_viewed_before_completed = 
t_viewed = pd.concat([ offer_interaction[offer_interaction.t_completed.isnull()], 
           offer_interaction[offer_interaction.t_viewed < offer_interaction.t_completed] ])[["person", "t_viewed"]]

In [ ]:
# replace t = nan(never viewed) with 1000(arbitrary long hour) 
t_viewed.t_viewed = t_viewed.t_viewed.replace(np.nan,1000)

# replace t
mask_over_120hour = t_viewed.t_viewed.astype(int) // 120 > 0

In [ ]:
mask_over_120hour.head(10)

In [ ]:
 t_viewed.loc[mask_over_120hour, "t_viewed"] = np.nan

In [ ]:
t_viewed.head(10)

In [ ]:
df_t_viewed = t_viewed.join(pd.get_dummies(t_viewed.t_viewed, dummy_na=True))
df_t_viewed.head(5)

In [ ]:
df_t_viewed2 = df_t_viewed.groupby("person").sum().iloc[:, 1:]
df_t_viewed2.columns = df_t_viewed2.columns.astype(str)
df_t_viewed2.head(5)

In [ ]:
df_t_viewed3 = df_t_viewed2.cumsum(axis=1)
df_t_viewed3 = df_t_viewed3.rename(columns = {"nan":"denom_viewed"})
df_t_viewed3.head()

In [ ]:
for column_name in df_t_viewed3.columns[:-1]:
        df_t_viewed3[column_name] = df_t_viewed3[column_name].astype(float) / df_t_viewed3.denom_viewed

In [ ]:
df_t_viewed3.head()

In [ ]:
# store this dataframe to the csv file
df_t_viewed3.to_csv("data/hour_vs_viewed.csv")